O objetivo deste exercício é utilizar outra base de dados para testar as amostragens e comparar os resultados <br>
<ol>
<li>Faça o download e carregue a base de dados credit_data.csv, que possui informações sobre empréstimos (se o cliente pagará ou não pagará o empréstimo)

<li>Teste cada uma das técnicas de amostragem, selecionando 1000 registros

<li>Para a amostragem estratificada, utilize o atributo c#default para separar as categorias

<li>No final, faça o comparativo da média utilizando os atributos age, income e loan

In [123]:
import pandas as pd
import numpy as np
import random

In [124]:
df_clientes = pd.read_csv('credit_data.csv')
df_clientes.tail()

,i#clientid,income,age,loan,c#default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


<h1>Amostragem aleatória simples</h1>

In [125]:
def amostragem_simples(dados: pd.DataFrame,num_amostras, seed):
    amostra_simples = dados.sample( n = num_amostras, random_state= seed)
    return amostra_simples

In [126]:
amostra_simples = amostragem_simples(df_clientes,1000,1)
print(len(amostra_simples))

1000


<h1> Amostragem sistemática


In [127]:
'''
opção 1 com lista
def amostragem_sistematica(dados: pd.DataFrame, num_amostras):
    random.seed(1)
    intervalo = len(dados)// num_amostras
    primeiro_num = random.randint(0,intervalo -1)
    amostra_sistematica = [x for x in range(primeiro_num,len(dados),intervalo)]
    return dados.iloc[amostra_sistematica]
'''

def amostragem_sistematica(dataset: pd.DataFrame, amostras,seed):
  random.seed(seed)
  intervalo = len(dataset) // amostras
  inicio = random.randint(0, intervalo - 1)
  indices = np.arange(inicio, len(dataset), step = intervalo)
  amostra_sistematica = dataset.iloc[indices]
  return amostra_sistematica

In [147]:
amostra_sistematica = amostragem_sistematica(df_clientes,1000,1)
amostra_sistematica

,i#clientid,income,age,loan,c#default,grupo
0,1,66155.925095,59.017015,8106.532131,0,0
2,3,57317.170063,63.108049,8020.953296,0,0
4,5,66952.688845,18.584336,8770.099235,1,0
6,7,48430.359613,26.809132,5722.581981,0,0
8,9,40654.892537,55.496853,4755.825280,0,0
...,...,...,...,...,...,...
1990,1991,34237.575419,34.101654,2658.090632,0,1
1992,1993,30803.806165,23.250084,623.024153,0,1
1994,1995,24254.700791,37.751622,2225.284643,0,1
1996,1997,69516.127573,23.162104,3503.176156,0,1


<h1>Amostragem por grupos


In [129]:
def amostragem_por_grupos(dados: pd.DataFrame, num_amostras, seed):
    random.seed(seed)
    num_grupos = len(dados) // num_amostras
    intervalos = len(dados) // num_grupos
    contagem = 0
    indice = 0
    grupo = []

    for _ in dados.iterrows(): #iterrows permite iterar o dataset:
        grupo.append(indice)
        contagem += 1
        if contagem > intervalos:
            contagem = 0
            indice += 1
    
    dados['grupo'] = grupo
    grupo_sorteado = random.randint(0,num_grupos - 1)
    amostra_por_grupo = dados[dados['grupo'] == grupo_sorteado]
    return amostra_por_grupo
        

In [130]:
amostra_grupos = amostragem_por_grupos(df_clientes, 1000, 1)
amostra_grupos.shape

(1001, 6)

<h1>Amostragem Estratificada <h1>

In [131]:
from sklearn.model_selection import StratifiedShuffleSplit
def amostragem_estratificada(data: pd.DataFrame, amostras,campo, seed):
    porcentagem = amostras / len(data)
    split = StratifiedShuffleSplit(test_size=porcentagem, random_state=seed)
    for _, y in split.split(data, data[campo]):
        df_y = data.iloc[y]
    return df_y


In [132]:
amostra_estratificada= amostragem_estratificada(df_clientes,1000,'c#default',5)
amostra_estratificada.shape

(1000, 6)

<h1> Amostragem Reservatorio

In [133]:
def amostragem_reservatorio(dataset, amostras):
  stream = []
  for i in range(len(dataset)):
    stream.append(i)

  i = 0
  tamanho = len(dataset)

  reservatorio = [0] * amostras
  for i in range(amostras):
    reservatorio[i] = stream[i]

  while (i < tamanho):
    j = random.randrange(i + 1)
    if (j < amostras):
      reservatorio[j] = stream[i]
    i += 1

  return dataset.iloc[reservatorio]

In [134]:
amostra_reservatorio = amostragem_reservatorio(df_clientes,1000)
amostra_reservatorio

,i#clientid,income,age,loan,c#default,grupo
0,1,66155.925095,59.017015,8106.532131,0,0
1649,1650,62219.037539,19.523983,5831.521429,0,1
2,3,57317.170063,63.108049,8020.953296,0,0
1114,1115,37887.549393,24.415726,5061.777831,1,1
1023,1024,68414.120777,51.046781,6154.052457,0,1
...,...,...,...,...,...,...
1476,1477,46024.144562,24.174516,4318.377722,0,1
996,997,49104.768240,35.538517,9452.217947,0,0
997,998,65776.232413,39.798191,2805.863745,0,0
998,999,36192.149452,21.402403,7236.173930,1,0


<h1>Comparação


In [177]:
dados_analisados = ['age','income','loan']
df_clientes_analise = df_clientes[dados_analisados]
age_base,income_base,loan_base = df_clientes_analise.mean()
df_clientes_analise.mean()

age          40.807559
income    45331.600018
loan       4444.369695
dtype: float64

In [182]:
df_amostra_simples = amostra_simples[dados_analisados]
print(df_amostra_simples.mean())
print()

age          40.959491
income    45422.483657
loan       4493.759721
dtype: float64

Diferença age: 0.1519321157356046
Diferença income: 90.88363928603212
Diferença loan: 49.3900259596885


In [166]:
df_amostra_sistematica = amostra_sistematica[dados_analisados]
df_amostra_sistematica.mean()

age          40.704049
income    44971.701285
loan       4381.951413
dtype: float64

In [169]:
df_amostra_grupos = amostra_grupos[dados_analisados]
df_amostra_grupos.mean()

age          41.043223
income    44846.749260
loan       4390.161494
dtype: float64

In [172]:
df_amostra_estratificada =  amostra_estratificada[dados_analisados]
df_amostra_estratificada.mean()

age          40.338648
income    46210.399212
loan       4495.031970
dtype: float64

In [175]:
df_amostra_reservatorio = amostra_reservatorio[dados_analisados]
df_amostra_reservatorio.mean()

age          40.572037
income    45111.194297
loan       4429.634971
dtype: float64

In [188]:
def impressao_diferenca(nome):
    print('Diferença age:', abs(nome['age'].mean() - age_base))
    print('Diferença income:', abs(nome['income'].mean() - income_base))
    print('Diferença loan:', abs(nome['loan'].mean() - loan_base))
    print()

print('Amostras simples: ')
impressao_diferenca(df_amostra_simples)
print('Amostra sistematica: ')
impressao_diferenca(df_amostra_sistematica)
print('Amostra estratificada:')
impressao_diferenca(df_amostra_estratificada)
print('Amostra grupos')
impressao_diferenca(df_amostra_grupos)
print('Amostra Reservatorio')
impressao_diferenca(df_amostra_reservatorio)

Amostras simples: 
Diferença age: 0.1519321157356046
Diferença income: 90.88363928603212
Diferença loan: 49.3900259596885

Amostra sistematica: 
Diferença age: 0.1035107148618053
Diferença income: 359.8987328761723
Diferença loan: 62.41828173807062

Amostra estratificada:
Diferença age: 0.46891122212718983
Diferença income: 878.79919427797
Diferença loan: 50.662275176941876

Amostra grupos
Diferença age: 0.23566373364572257
Diferença income: 484.85075793183205
Diferença loan: 54.208200944053715

Amostra Reservatorio
Diferença age: 0.23552229632121424
Diferença income: 220.4057204478886
Diferença loan: 14.734723317888893

